In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import time
import operator
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [2]:
# Create DataFrames from CSV files
t0 = time.time()
filtered_lyric_TF_df = pd.read_csv('Data/filtered_lyric_TF.csv')
filtered_lyric_TF_df = filtered_lyric_TF_df.drop([0,1])
filtered_lyric_TF_df.index = filtered_lyric_TF_df.index - 2
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
filtered_lyric_TF_df.head(3)

Run time: 38.54279279708862 seconds


,song_name,artist_name,category_name,category_id,spotify_popularity,genre_list,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_mode,...,rolex,contains,ashton,anywhere,recapture,nick,practice,compose,starin,chill
0,willow,Taylor Swift,pop,8.0,93.0,"['dance', 'pop']",0.392,0.574,7.0,1.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Stay Next To Me (with Chelsea Cutler),Quinn XCII,pop,8.0,78.0,"['indie', 'pop', 'electropop']",0.581,0.584,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,WITHOUT YOU,The Kid LAROI,pop,8.0,95.0,['australian'],0.662,0.413,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
# Create a dictionary of category names and ids
cat_name_id = {}
category_list = ['blues', 'classical', 'country', 'funk', 'hiphop', 'indie_alt', 'jazz', 
                 'metal', 'pop', 'punk', 'rnb', 'rock', 'romance', 'soul']
for cat in category_list:
    cat_id = list(filtered_lyric_TF_df[filtered_lyric_TF_df['category_name']==cat]['category_id'])[0]
    cat_name_id[cat] = cat_id
    print(cat, cat_id)

blues 0.0
classical 1.0
country 2.0
funk 3.0
hiphop 4.0
indie_alt 5.0
jazz 6.0
metal 7.0
pop 8.0
punk 9.0
rnb 10.0
rock 11.0
romance 12.0
soul 13.0


In [4]:
# Create a genre lists column
t0 = time.time()
genre_lists = []
for index, row in filtered_lyric_TF_df.iterrows():
    genres = row['genre_list'].replace('[', '').replace(']', '').replace("'", '').split(', ')
    genre_lists.append(genres)
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
len(genre_lists)

Run time: 51.82529616355896 seconds


8041

In [5]:
# Replace the genre_list column
filtered_lyric_TF_df['genre_list'] = genre_lists

In [6]:
# Print how many songs there are in each category
for cat in category_list:
    song_count = len(filtered_lyric_TF_df[filtered_lyric_TF_df['category_name']==cat])
    print(f'There are {song_count} songs in the {cat} category.')

There are 424 songs in the blues category.
There are 95 songs in the classical category.
There are 1257 songs in the country category.
There are 282 songs in the funk category.
There are 683 songs in the hiphop category.
There are 660 songs in the indie_alt category.
There are 198 songs in the jazz category.
There are 898 songs in the metal category.
There are 693 songs in the pop category.
There are 569 songs in the punk category.
There are 362 songs in the rnb category.
There are 1229 songs in the rock category.
There are 280 songs in the romance category.
There are 411 songs in the soul category.


In [7]:
# Get the most popular genres for each category
t0 = time.time()
category_genres = {}
for category in category_list:
    genres = []
    for index, row in filtered_lyric_TF_df[filtered_lyric_TF_df['category_name']==category].iterrows():
        genres.extend(row['genre_list'])
    unique_genres = list(set(genres))
    genre_counts = []
    for genre in unique_genres:
        genre_counts.append((genre, genres.count(genre)))
    category_genres[category] = sorted(genre_counts, key = lambda x: x[1], reverse=True)[0:5]
    print(category, category_genres[category])
t1 = time.time()
print(f'Run time: {t1-t0} seconds')

blues [('blues', 260), ('electric', 190), ('modern', 168), ('traditional', 146), ('classic', 106)]
classical [('classical', 17), ('operatic', 14), ('dance', 12), ('', 9), ('pop', 9)]
country [('country', 977), ('contemporary', 847), ('modern', 277), ('pop', 88), ('nashville', 78)]
funk [('funk', 133), ('soul', 118), ('motown', 97), ('quiet', 82), ('disco', 79)]
hiphop [('rap', 346), ('hip', 305), ('pop', 228), ('trap', 192), ('southern', 143)]
indie_alt [('indie', 291), ('alternative', 181), ('new', 155), ('modern', 148), ('art', 144)]
jazz [('vocal', 93), ('contemporary', 83), ('jazz', 79), ('adult', 64), ('neo', 38)]
metal [('metal', 368), ('alternative', 286), ('nu', 222), ('rock', 221), ('hard', 207)]
pop [('pop', 344), ('indie', 180), ('dance', 164), ('post-teen', 129), ('electropop', 129)]
punk [('pop', 195), ('punk', 138), ('skate', 94), ('alternative', 84), ('melodic', 72)]
rnb [('pop', 219), ('r&b', 213), ('urban', 200), ('dance', 169), ('hip', 136)]
rock [('rock', 638), ('mod

In [8]:
# Find indeces to keep and indeces to remove
t0 = time.time()
cat_genre_dict = {'pop': ['pop'], 
                  'hiphop': ['rap', 'hip', 'trap'],
                  'metal': ['metal'], 
                  'rock': ['classic'], 
                  'jazz': ['jazz'], 
                  'rnb': ['r&b'], 
                  'romance': ['soft'], 
                  'soul': ['soul'], 
                  'indie_alt': ['indie', 'alternative'],
                  'classical': ['classical', 'operatic', 'dance'], 
                  'blues': ['modern', 'blues'],
                  'funk': ['funk', 'disco'], 
                  'punk': ['punk'],
                  'country': ['country']} 
indeces_to_keep = []
indeces_to_remove = []
for index, row in filtered_lyric_TF_df.iterrows():
    keep = False
    category = row['category_name']
    for genre in cat_genre_dict[category]:
        if genre in row['genre_list']:
            keep = True
            break
    if keep == True:
        indeces_to_keep.append(index)
    else:
        indeces_to_remove.append(index)
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
print(len(indeces_to_keep))
len(indeces_to_remove)

Run time: 37.566579818725586 seconds
3990


4051

In [9]:
# Create non destructive copy and remove the unnecessary indeces
filtered_lyric_TF_df2 = filtered_lyric_TF_df.drop(indeces_to_remove)
filtered_lyric_TF_df2.head(3)

Run time: 0.36324000358581543 seconds


,song_name,artist_name,category_name,category_id,spotify_popularity,genre_list,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_mode,...,rolex,contains,ashton,anywhere,recapture,nick,practice,compose,starin,chill
0,willow,Taylor Swift,pop,8.0,93.0,"[dance, pop]",0.392,0.574,7.0,1.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Stay Next To Me (with Chelsea Cutler),Quinn XCII,pop,8.0,78.0,"[indie, pop, electropop]",0.581,0.584,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,r u ok,Tate McRae,pop,8.0,80.0,"[dance, pop, electropop, post-teen]",0.666,0.593,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# Remove categories
filtered_lyric_TF_df2 = filtered_lyric_TF_df2[filtered_lyric_TF_df2['category_name']!='romance']
filtered_lyric_TF_df2 = filtered_lyric_TF_df2[filtered_lyric_TF_df2['category_name']!='punk']
filtered_lyric_TF_df2 = filtered_lyric_TF_df2[filtered_lyric_TF_df2['category_name']!='blues']
filtered_lyric_TF_df2 = filtered_lyric_TF_df2[filtered_lyric_TF_df2['category_name']!='indie_alt']
len(filtered_lyric_TF_df2)

3082

In [11]:
# Print how many songs there are in each category 
category_list = ['blues', 'classical', 'country', 'funk', 'hiphop', 'indie_alt', 'jazz', 
                 'metal', 'pop', 'punk', 'rnb', 'rock', 'romance', 'soul']
for cat in category_list:
    song_count = len(filtered_lyric_TF_df2[filtered_lyric_TF_df2['category_name']==cat])
    print(f'There are {song_count} songs in the {cat} category.')

There are 0 songs in the blues category.
There are 38 songs in the classical category.
There are 977 songs in the country category.
There are 163 songs in the funk category.
There are 447 songs in the hiphop category.
There are 0 songs in the indie_alt category.
There are 79 songs in the jazz category.
There are 368 songs in the metal category.
There are 344 songs in the pop category.
There are 0 songs in the punk category.
There are 213 songs in the rnb category.
There are 245 songs in the rock category.
There are 0 songs in the romance category.
There are 208 songs in the soul category.


In [12]:
# Define the features set
X = filtered_lyric_TF_df2.copy()
X = X.drop(['song_name', 'artist_name', 'category_name', 'category_id', 'genre_list'], axis=1)
X.head(3)

,spotify_popularity,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_mode,audio_ft_speechiness,audio_ft_acousticness,audio_ft_instrumentalness,audio_ft_liveness,audio_ft_valence,...,rolex,contains,ashton,anywhere,recapture,nick,practice,compose,starin,chill
0,93.0,0.392,0.574,7.0,1.0,0.1700,0.8330,0.00179,0.145,0.529,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
1,78.0,0.581,0.584,2.0,1.0,0.2840,0.0805,0.00000,0.366,0.756,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,80.0,0.666,0.593,2.0,1.0,0.0373,0.3180,0.00000,0.414,0.329,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
# Define the target set
y = filtered_lyric_TF_df2['category_id'].ravel()
y[:5]

array([8., 8., 8., 8., 8.])

In [14]:
# Split into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(2311, 12081)
(771, 12081)
(2311,)
(771,)


In [15]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the Standard Scaler with the training data
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Fitting the Model

In [16]:
# Create a random forest classifier
rf_model = RandomForestClassifier(n_estimators=500, random_state=78)

In [17]:
# Fit the model
t0 = time.time()
rf_model = rf_model.fit(X_train_scaled, y_train)
t1 = time.time()
print(f'Run time: {t1-t0} seconds')

Run time: 30.88883924484253 seconds


# Making Predictions

In [18]:
# Make predictions using the testing data
predictions = rf_model.predict(X_test_scaled)
predictions

array([ 2.,  7.,  2.,  4.,  2.,  2., 11.,  2.,  2.,  2.,  3.,  8.,  4.,
        4.,  2.,  2.,  2.,  2.,  2.,  7.,  2.,  2.,  7.,  4.,  2.,  2.,
        7.,  2.,  2.,  2.,  8.,  4.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  4.,  2., 11.,  2., 13.,  4.,  2.,  7., 10.,  2.,  2.,
        7.,  2.,  2.,  7.,  4.,  2.,  8.,  7.,  2.,  2.,  2.,  2.,  4.,
        2.,  2.,  7.,  2.,  4.,  7.,  2.,  2.,  4.,  4.,  2.,  7.,  2.,
        2.,  2.,  8.,  2.,  2.,  2.,  2.,  7.,  7.,  4.,  2.,  4.,  4.,
        2.,  2.,  4.,  2.,  4.,  2.,  2.,  7.,  7.,  4.,  4., 13., 11.,
        2.,  2.,  2.,  4., 10.,  2.,  2.,  2.,  2.,  4.,  8.,  2.,  2.,
        2.,  2., 13.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  4.,  4.,  4.,
        7.,  2.,  8.,  2.,  2.,  4.,  2.,  4.,  3.,  2.,  2.,  4.,  4.,
        2.,  2.,  4.,  2.,  2.,  2.,  2.,  2.,  8.,  2.,  4.,  2.,  7.,
        2.,  2.,  2.,  2.,  2.,  7.,  2.,  7.,  2.,  2.,  4.,  2.,  2.,
        2.,  2.,  2.,  7.,  2.,  2.,  2.,  8.,  2.,  7.,  4.,  2

# Model Evaluation

In [19]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from cm
cm_df = pd.DataFrame(
    cm, index=['Actual classical', 'Actual country', 'Actual funk',
               'Actual hiphop', 'Actual jazz', 'Actual metal',
               'Actual pop', 'Actual rnb', 'Actual rock', 'Actual soul'],
        columns=['Predicted classical', 'Predicted country',
                 'Predicted funk', 'Predicted hiphop',
                 'Predicted jazz', 'Predicted metal', 'Predicted pop',
                 'Predicted rnb', 'Predicted rock', 'Predicted soul'])

# Calculate the accuracy score
acc_score = accuracy_score(y_test, predictions)

In [20]:
# Display the results
print('Confusion Matrix')
display(cm_df)
print(f'Accuracy Score: {acc_score}')
print('Classification Report')
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted classical,Predicted country,Predicted funk,Predicted hiphop,Predicted jazz,Predicted metal,Predicted pop,Predicted rnb,Predicted rock,Predicted soul
Actual classical,0,5,0,0,0,2,0,0,0,0
Actual country,0,251,0,0,0,0,0,0,0,0
Actual funk,0,17,5,2,0,2,0,0,0,2
Actual hiphop,0,5,0,112,0,0,0,0,0,0
Actual jazz,0,22,0,2,2,0,0,0,0,0
Actual metal,0,10,0,0,0,73,0,0,0,0
Actual pop,0,74,0,6,0,0,18,1,0,0
Actual rnb,0,27,1,14,0,0,1,7,0,0
Actual rock,0,50,0,0,0,5,0,0,5,0
Actual soul,0,34,3,0,1,1,0,1,0,10


Accuracy Score: 0.6264591439688716
Classification Report
              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00         7
         2.0       0.51      1.00      0.67       251
         3.0       0.56      0.18      0.27        28
         4.0       0.82      0.96      0.89       117
         6.0       0.67      0.08      0.14        26
         7.0       0.88      0.88      0.88        83
         8.0       0.95      0.18      0.31        99
        10.0       0.78      0.14      0.24        50
        11.0       1.00      0.08      0.15        60
        13.0       0.83      0.20      0.32        50

    accuracy                           0.63       771
   macro avg       0.70      0.37      0.39       771
weighted avg       0.73      0.63      0.55       771



//anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
# Calculate the feature importance in the Random Forest Model
importances = rf_model.feature_importances_
importances

array([1.09186117e-02, 1.53084232e-02, 1.69982535e-02, ...,
       1.97472283e-05, 3.50137975e-05, 1.59013423e-04])

In [22]:
# Sort the features by importance
sorted_features = sorted(zip(importances, X.columns), reverse=True)
sorted_features

[(0.020660607493541292, 'audio_ft_acousticness'),
 (0.017677339549131033, 'audio_ft_speechiness'),
 (0.016998253456920173, 'audio_ft_energy'),
 (0.015308423214145606, 'audio_ft_danceability'),
 (0.013794079598340853, 'audio_ft_duration_ms'),
 (0.013145521242518787, 'audio_ft_instrumentalness'),
 (0.011247991860326937, 'audio_ft_valence'),
 (0.010918611746909203, 'spotify_popularity'),
 (0.006655513192345003, 'audio_ft_mode'),
 (0.006231388350745676, 'audio_ft_liveness'),
 (0.005903024899480157, 'love'),
 (0.005829241788281269, 'audio_ft_tempo'),
 (0.005745204688306863, '?'),
 (0.005246640070140494, 'baby'),
 (0.005206655827312094, 'nigga'),
 (0.005185175645076619, 'shit'),
 (0.005177316246312349, 'im'),
 (0.005137587137782303, 'oh'),
 (0.005121235571600685, 'like'),
 (0.005109396023086372, 'niggas'),
 (0.0048590697461831575, 'know'),
 (0.004845597464357811, 'aint'),
 (0.004537932329641146, 'bitch'),
 (0.0045364836104217505, 'fuck'),
 (0.004245790085297161, 'dont'),
 (0.0041745094205231

# Dropping Lower Ranked Features

In [26]:
# Create a dictionary of features and accuracy scores
features_accuracy = {}
feature_percents = [0.2, 0.3, 0.4]
for percent in feature_percents:
    # Define the features and target sets
    important_features = []
    sum = 0
    count = 0
    while sum < percent:
        sum = sum + sorted_features[count][0]
        important_features.append(sorted_features[count][1])
        count+=1
    X = filtered_lyric_TF_df2[important_features]
    y = filtered_lyric_TF_df2['category_id'].ravel()
    # Split into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)
    # Scale the data
    scaler = StandardScaler()
    X_scaler = scaler.fit(X_train)
    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)
    # Fit the random forest model
    rf_model = RandomForestClassifier(n_estimators=500, random_state=78)
    t0 = time.time()
    rf_model = rf_model.fit(X_train_scaled, y_train)
    t1 = time.time()
    print(f'Run time to fit model with {percent*100}% of total features: {t1-t0} seconds')
    # Make predictions
    predictions = rf_model.predict(X_test_scaled)
    # Evaluations
    cm = confusion_matrix(y_test, predictions)
    cm_df = pd.DataFrame(
        cm, index=['Actual classical', 'Actual country', 'Actual funk',
                   'Actual hiphop', 'Actual jazz', 'Actual metal',
                   'Actual pop', 'Actual rnb', 'Actual rock', 'Actual soul'],
            columns=['Predicted classical', 'Predicted country',
                     'Predicted funk', 'Predicted hiphop',
                     'Predicted jazz', 'Predicted metal', 'Predicted pop',
                     'Predicted rnb', 'Predicted rock', 'Predicted soul'])
    acc_score = accuracy_score(y_test, predictions)
    print('Confusion Matrix')
    display(cm_df)
    print(f'Accuracy Score: {acc_score}')
    print('Classification Report')
    print(classification_report(y_test, predictions))
    features_accuracy[percent] = acc_score

Run time to fit model with 20.0% of total features: 3.8090760707855225 seconds
Confusion Matrix


,Predicted classical,Predicted country,Predicted funk,Predicted hiphop,Predicted jazz,Predicted metal,Predicted pop,Predicted rnb,Predicted rock,Predicted soul
Actual classical,3,1,0,0,0,2,0,0,1,0
Actual country,0,235,2,0,1,3,5,2,2,1
Actual funk,0,7,16,0,1,1,0,1,0,2
Actual hiphop,0,2,0,108,0,0,5,2,0,0
Actual jazz,2,9,1,2,11,0,0,0,1,0
Actual metal,0,6,0,0,0,73,1,0,3,0
Actual pop,0,32,1,8,0,0,55,3,0,0
Actual rnb,0,6,4,10,0,0,8,21,1,0
Actual rock,2,26,5,1,0,7,0,0,17,2
Actual soul,0,31,5,0,1,0,1,3,4,5


Accuracy Score: 0.7055771725032426
Classification Report
              precision    recall  f1-score   support

         1.0       0.43      0.43      0.43         7
         2.0       0.66      0.94      0.78       251
         3.0       0.47      0.57      0.52        28
         4.0       0.84      0.92      0.88       117
         6.0       0.79      0.42      0.55        26
         7.0       0.85      0.88      0.86        83
         8.0       0.73      0.56      0.63        99
        10.0       0.66      0.42      0.51        50
        11.0       0.59      0.28      0.38        60
        13.0       0.50      0.10      0.17        50

    accuracy                           0.71       771
   macro avg       0.65      0.55      0.57       771
weighted avg       0.70      0.71      0.67       771

Run time to fit model with 30.0% of total features: 3.8872179985046387 seconds
Confusion Matrix


,Predicted classical,Predicted country,Predicted funk,Predicted hiphop,Predicted jazz,Predicted metal,Predicted pop,Predicted rnb,Predicted rock,Predicted soul
Actual classical,3,1,0,0,0,2,0,0,1,0
Actual country,0,242,1,0,2,1,5,0,0,0
Actual funk,0,8,15,0,0,1,0,1,1,2
Actual hiphop,0,3,0,111,0,0,3,0,0,0
Actual jazz,2,9,2,2,9,0,0,0,2,0
Actual metal,0,5,0,0,0,75,1,0,2,0
Actual pop,0,32,1,6,0,0,56,3,1,0
Actual rnb,0,6,2,16,0,0,8,18,0,0
Actual rock,2,27,5,0,0,7,0,0,17,2
Actual soul,1,29,4,0,2,0,1,2,1,10


Accuracy Score: 0.7211413748378729
Classification Report
              precision    recall  f1-score   support

         1.0       0.38      0.43      0.40         7
         2.0       0.67      0.96      0.79       251
         3.0       0.50      0.54      0.52        28
         4.0       0.82      0.95      0.88       117
         6.0       0.69      0.35      0.46        26
         7.0       0.87      0.90      0.89        83
         8.0       0.76      0.57      0.65        99
        10.0       0.75      0.36      0.49        50
        11.0       0.68      0.28      0.40        60
        13.0       0.71      0.20      0.31        50

    accuracy                           0.72       771
   macro avg       0.68      0.55      0.58       771
weighted avg       0.73      0.72      0.69       771

Run time to fit model with 40.0% of total features: 4.862344980239868 seconds
Confusion Matrix


,Predicted classical,Predicted country,Predicted funk,Predicted hiphop,Predicted jazz,Predicted metal,Predicted pop,Predicted rnb,Predicted rock,Predicted soul
Actual classical,3,1,0,0,0,2,0,0,1,0
Actual country,0,243,1,0,2,2,3,0,0,0
Actual funk,0,7,14,1,0,0,0,1,3,2
Actual hiphop,0,2,0,112,0,0,3,0,0,0
Actual jazz,2,10,1,2,9,0,0,0,2,0
Actual metal,0,5,0,0,0,76,1,0,1,0
Actual pop,0,39,0,7,0,1,49,2,1,0
Actual rnb,0,7,3,16,0,0,5,19,0,0
Actual rock,2,32,4,0,0,7,0,0,15,0
Actual soul,0,29,5,0,2,0,1,2,3,8


Accuracy Score: 0.7107652399481194
Classification Report
              precision    recall  f1-score   support

         1.0       0.43      0.43      0.43         7
         2.0       0.65      0.97      0.78       251
         3.0       0.50      0.50      0.50        28
         4.0       0.81      0.96      0.88       117
         6.0       0.69      0.35      0.46        26
         7.0       0.86      0.92      0.89        83
         8.0       0.79      0.49      0.61        99
        10.0       0.79      0.38      0.51        50
        11.0       0.58      0.25      0.35        60
        13.0       0.80      0.16      0.27        50

    accuracy                           0.71       771
   macro avg       0.69      0.54      0.57       771
weighted avg       0.72      0.71      0.68       771



In [27]:
# Print feature percents and accuracy scores
for percent in features_accuracy:
    acc_score = features_accuracy[percent]
    print(f'Accuracy Score for {percent*100}% of total features: {acc_score}')

Accuracy Score for 20.0% of total features: 0.7055771725032426
Accuracy Score for 30.0% of total features: 0.7211413748378729
Accuracy Score for 40.0% of total features: 0.7107652399481194


In [28]:
# Get the highest accuracy score
best_percent = max(features_accuracy, key=features_accuracy.get)
best_acc = features_accuracy[best_percent]
print(f'The highest accuracy score of {best_acc} was achieved using {best_percent*100}% of all features.')

The highest accuracy score of 0.7211413748378729 was achieved using 30.0% of all features.
